# Vision&Perception Project
# SGCN:Sparse Graph Convolution Network for Pedestrian Trajectory Prediction

## 1. Import Libraries

In [1]:
import os 

import glob
import copy
import pickle
import argparse
import warnings
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import tensorflow_probability as tfp
import datetime

from utils import *
from model import *
from metrics import *

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

## 2. Settings
### 2.1 Setting GPU training

In [2]:
print('Tensorflow Version: ', tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.device('/device:GPU:0')

Tensorflow Version:  2.3.0
Num GPUs Available:  1


### 2.2 Setting Parameters

In [3]:
# Setting parameters
parser = argparse.ArgumentParser()

parser.add_argument('--obs_len', type=int, default=8)
parser.add_argument('--pred_len', type=int, default=12)
parser.add_argument('--dataset', default='data', help='eth,hotel,univ,zara1,zara2')
# Training specifc parameters
parser.add_argument('--batch_size', type=int, default=128, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=20, help='number of epochs')
parser.add_argument('--clip_grad', type=float, default=10, help='gadient clipping')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum of lr')
parser.add_argument('--weight_decay', type=float, default=0.0001, help='weight_decay on l2 reg')
parser.add_argument('--lr_sh_rate', type=int, default=100, help='number of steps to drop the lr')
parser.add_argument('--milestones', type=int, default=[50, 100], help='number of steps to drop the lr')
parser.add_argument('--use_lrschd', action="store_true", default=True, help='Use lr rate scheduler')
parser.add_argument('--tag', default='sgcn', help='personal tag for the model ')
parser.add_argument('--gpu_num', default="0", type=str)

args = parser.parse_args(args=[])

print("Training initiating....")
print(args)

Training initiating....
Namespace(batch_size=128, clip_grad=10, dataset='data', gpu_num='0', lr=0.001, lr_sh_rate=100, milestones=[50, 100], momentum=0.9, num_epochs=20, obs_len=8, pred_len=12, tag='sgcn', use_lrschd=True, weight_decay=0.0001)


### 2.3 Setting Loss Function

In [4]:
# computing loss
def graph_loss(V_pred, V_target):
    return bivariate_loss(V_pred, V_target)

## 3. Loading Training & Validation Dataset

In [5]:
obs_seq_len = args.obs_len                        # obs_len=8
pred_seq_len = args.pred_len                      # pred_len=12
data_set = './dataset/data/'

# Loading training data
dset_train = TrajectoryDataset(
    data_set + 'train1/',
    obs_len=obs_seq_len,
    pred_len=pred_seq_len,
    skip=1)
loader_train = np.asarray(dset_train)

# Loading validation data
dset_val = TrajectoryDataset(
    data_set + 'valid1/',
    obs_len=obs_seq_len,
    pred_len=pred_seq_len,
    skip=1)
loader_val = np.asarray(dset_val)

 29%|██████████████████████▏                                                      | 934/3244 [00:00<00:00, 9335.89it/s]

Processing Data .....


100%|█████████████████████████████████████████████████████████████████████████████| 895/895 [00:00<00:00, 11053.05it/s]

Processing Data .....


## 4. Preparing Model

### 4.1 Specifying GPU Training

In [6]:
print('Training started ...')
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_num

Training started ...


### 4.2 Instantiating Model

In [7]:
model = TrajectoryModel(number_asymmetric_conv_layer=7, embedding_dims=64, number_gcn_layers=1, dropout=0,
                        obs_len=8, pred_len=12, n_tcn=5, out_dims=5)

### 4.3 Initializing Model

In [8]:
tf.get_logger().setLevel('INFO')
for cnt, batch in enumerate(loader_train):
    obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, loss_mask, V_obs, V_tr = batch
    V_obs = V_obs[np.newaxis, :]
    identity_spatial = tf.ones((V_obs.shape[1], V_obs.shape[2], V_obs.shape[2])) * tf.eye(V_obs.shape[2])  # [obs_len N N]
    identity_temporal = tf.ones((V_obs.shape[2], V_obs.shape[1], V_obs.shape[1])) * tf.eye(V_obs.shape[1])  # [N obs_len obs_len]
    identity = np.array([identity_spatial, identity_temporal])
    V_pred = model(V_obs, identity)
    break


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

### 4.4 Initializing Optimizer

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

### 4.5 Using Tensorboard

In [10]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
valid_log_dir = 'logs/gradient_tape/' + current_time + '/valid'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

train_loss = tf.keras.metrics.Mean('Train Loss', dtype=tf.float32)
valid_loss = tf.keras.metrics.Mean('Valid Loss', dtype=tf.float32)
ADE_loss = tf.keras.metrics.Mean('ADE', dtype=tf.float32)
FDE_loss = tf.keras.metrics.Mean('FDE', dtype=tf.float32)

### 4.6 Defining Train, Valid & Test function

In [11]:
def train(epoch, model, optimizer, loader_train):
    global metrics, constant_metrics
    
    batch_count = 0
    is_fst_loss = True
    flag = False
    loader_len = len(loader_train)
    turn_point = int(loader_len / args.batch_size) * args.batch_size + loader_len % args.batch_size - 1
    
    loss_sum_list = []
    V_obs_list = []
    V_tr_list = []
    identity_list = []
    
    for cnt, batch in enumerate(loader_train):
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, \
        loss_mask, V_obs, V_tr = batch

        obs_traj = obs_traj[np.newaxis, :]
        pred_traj_gt = pred_traj_gt[np.newaxis, :]
        obs_traj_rel = obs_traj_rel[np.newaxis, :]
        pred_traj_gt_rel = pred_traj_gt_rel[np.newaxis, :]
        non_linear_ped = non_linear_ped[np.newaxis, :]
        loss_mask = loss_mask[np.newaxis, :]
        V_obs = V_obs[np.newaxis, :]
        V_tr = V_tr[np.newaxis, :]

        identity_spatial = tf.ones((V_obs.shape[1], V_obs.shape[2], V_obs.shape[2])) * \
                           tf.eye(V_obs.shape[2])
        identity_temporal = tf.ones((V_obs.shape[2], V_obs.shape[1], V_obs.shape[1])) * \
                            tf.eye(V_obs.shape[1])
        identity = [identity_spatial, identity_temporal]

        if (cnt + 1) % 128 != 0:
            V_obs_list.append(V_obs)
            V_tr_list.append(V_tr)
            identity_list.append(identity)
        else:
            is_first_loss = True
            batch_count += 1
            with tf.GradientTape() as tape:
                tape.watch(model.trainable_weights)
                V_list = [(model(V_obs, identity),tf.cast(tf.squeeze(V_tr), dtype = tf.float32)) for V_obs, V_tr, identity in zip(V_obs_list,V_tr_list,identity_list)]
                loss_ = sum([graph_loss(item[0], item[1]) for item in V_list])/124
            grads = tape.gradient(loss_, model.trainable_weights)
            
            V_obs_list = []
            V_tr_list = []
            identity_list = []
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
            train_loss(loss_)
            with train_summary_writer.as_default():
                tf.summary.scalar('train_loss', train_loss.result(), step=epoch * 25 + batch_count)

            print('TRAIN:', '\t cnt:', cnt, '\t Loss:', loss_)

def valid(epoch, model, loader_val):
    global metrics, constant_metrics
#     model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point = int(loader_len / args.batch_size) * args.batch_size + loader_len % args.batch_size - 1

    for cnt, batch in enumerate(loader_val):

        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, \
        loss_mask, V_obs, V_tr = batch
    
        obs_traj = obs_traj[np.newaxis, :]
        pred_traj_gt = pred_traj_gt[np.newaxis, :]
        obs_traj_rel = obs_traj_rel[np.newaxis, :]
        pred_traj_gt_rel = pred_traj_gt_rel[np.newaxis, :]
        non_linear_ped = non_linear_ped[np.newaxis, :]
        loss_mask = loss_mask[np.newaxis, :]
        V_obs = V_obs[np.newaxis, :]
        V_tr = V_tr[np.newaxis, :]

        identity_spatial = tf.ones((V_obs.shape[1], V_obs.shape[2], V_obs.shape[2])) * tf.eye(V_obs.shape[2])
        identity_temporal = tf.ones((V_obs.shape[2], V_obs.shape[1], V_obs.shape[1])) * tf.eye(V_obs.shape[1])
        identity = [identity_spatial, identity_temporal]

        V_pred = model(V_obs, identity)  # A_obs <8, #, #>

        V_tr = tf.cast(tf.squeeze(V_tr), dtype = tf.float32)

        if (cnt + 1) % 128 != 0:
            l = graph_loss(V_pred, V_tr)

            if is_fst_loss:
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss / args.batch_size
            
            batch_count += 1
            
            valid_loss(loss)
            with valid_summary_writer.as_default():
                tf.summary.scalar('valid_loss', valid_loss.result(), step=epoch * 6 + batch_count)
            is_fst_loss = True
            print('VALID:', '\t cnt:', cnt, '\t Loss:', loss)

def test(model, loader_test, KSTEPS=20):
    raw_data_dict = {}
    ade_bigls = []
    fde_bigls = []

    step =0
    for batch in tqdm(loader_test):
        step += 1
        
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, loss_mask, V_obs, V_tr = batch
        
        obs_traj = obs_traj[np.newaxis, :]
        pred_traj_gt = pred_traj_gt[np.newaxis, :]
        obs_traj_rel = obs_traj_rel[np.newaxis, :]
        pred_traj_gt_rel = pred_traj_gt_rel[np.newaxis, :]
        non_linear_ped = non_linear_ped[np.newaxis, :]
        loss_mask = loss_mask[np.newaxis, :]
        V_obs = V_obs[np.newaxis, :]
        V_tr = V_tr[np.newaxis, :]

        identity_spatial = tf.ones((V_obs.shape[1], V_obs.shape[2], V_obs.shape[2])) * tf.eye(V_obs.shape[2])
        identity_temporal = tf.ones((V_obs.shape[2], V_obs.shape[1], V_obs.shape[1])) * tf.eye(V_obs.shape[1])
        identity = [identity_spatial, identity_temporal]
        
        V_pred = model(V_obs, identity)
        V_tr = tf.cast(tf.squeeze(V_tr), dtype = tf.float32)
        
        num_of_objs = obs_traj_rel.shape[1]
        V_pred, V_tr = V_pred[:, :num_of_objs, :], V_tr[:, :num_of_objs, :]
        
        sx = tf.math.exp(V_pred[:,:,2]) #sx
        sy = tf.math.exp(V_pred[:,:,3]) #sy
        corr = tf.math.tanh(V_pred[:,:,4]) #corr
        
        cov = np.zeros((V_pred.shape[0],V_pred.shape[1],2,2))
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        cov = tf.convert_to_tensor(cov, dtype=tf.float32)
        mean = V_pred[:,:,0:2]
        
        mvnormal = tfp.distributions.MultivariateNormalFullCovariance(mean, cov)
        
        ade_ls = {}
        fde_ls = {}
        V_x = seq_to_nodes(tf.identity(obs_traj))
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_obs[:,:,:,:2])), tf.identity(V_x[0,:,:]))
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_tr)),tf.identity(V_x[-1,:,:]))
        
        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []
        
        for n in range(num_of_objs):
            ade_ls[n]=[]
            fde_ls[n]=[]
        
        for k in range(KSTEPS):

            V_pred = mvnormal.sample()
    
            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_pred)), tf.identity(V_x[-1,:,:]))

            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))

            for n in range(num_of_objs):
                pred = []
                target = []
                obsrvs = []
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)
                
                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))
                

                   
        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))

        ADE_loss(sum(ade_bigls)/len(ade_bigls))
        FDE_loss(sum(fde_bigls)/len(fde_bigls))
        with test_summary_writer.as_default():
            tf.summary.scalar('ADE_loss', ADE_loss.result(), step=step)
            tf.summary.scalar('FDE_loss', FDE_loss.result(), step=step)
        ADE_loss.reset_states()
        FDE_loss.reset_states()
    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_,fde_,raw_data_dict

## 5. Training

In [12]:
for epoch in range(args.num_epochs):
    print("Epoch: ", epoch)
    if epoch == 10:
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
    
    train(epoch, model, optimizer, loader_train)
    train_loss.reset_states()
    valid(epoch, model, loader_val)
    valid_loss.reset_states()

Epoch:  0
TRAIN: 	 cnt: 127 	 Loss: tf.Tensor(1.8883142, shape=(), dtype=float32)
TRAIN: 	 cnt: 255 	 Loss: tf.Tensor(1.8822917, shape=(), dtype=float32)
TRAIN: 	 cnt: 383 	 Loss: tf.Tensor(1.8799993, shape=(), dtype=float32)
TRAIN: 	 cnt: 511 	 Loss: tf.Tensor(1.8763658, shape=(), dtype=float32)
TRAIN: 	 cnt: 639 	 Loss: tf.Tensor(1.8756995, shape=(), dtype=float32)
TRAIN: 	 cnt: 767 	 Loss: tf.Tensor(1.8794144, shape=(), dtype=float32)
TRAIN: 	 cnt: 895 	 Loss: tf.Tensor(1.8713725, shape=(), dtype=float32)
TRAIN: 	 cnt: 1023 	 Loss: tf.Tensor(1.86997, shape=(), dtype=float32)
TRAIN: 	 cnt: 1151 	 Loss: tf.Tensor(1.8652644, shape=(), dtype=float32)
TRAIN: 	 cnt: 1279 	 Loss: tf.Tensor(1.863199, shape=(), dtype=float32)
TRAIN: 	 cnt: 1407 	 Loss: tf.Tensor(1.8606985, shape=(), dtype=float32)
TRAIN: 	 cnt: 1535 	 Loss: tf.Tensor(1.8585362, shape=(), dtype=float32)
TRAIN: 	 cnt: 1663 	 Loss: tf.Tensor(1.8584445, shape=(), dtype=float32)
TRAIN: 	 cnt: 1791 	 Loss: tf.Tensor(1.8577721, sha

TRAIN: 	 cnt: 2559 	 Loss: tf.Tensor(-1.833419, shape=(), dtype=float32)
TRAIN: 	 cnt: 2687 	 Loss: tf.Tensor(-2.0158515, shape=(), dtype=float32)
TRAIN: 	 cnt: 2815 	 Loss: tf.Tensor(-1.8645903, shape=(), dtype=float32)
TRAIN: 	 cnt: 2943 	 Loss: tf.Tensor(-1.9747514, shape=(), dtype=float32)
TRAIN: 	 cnt: 3071 	 Loss: tf.Tensor(-1.8362759, shape=(), dtype=float32)
TRAIN: 	 cnt: 3199 	 Loss: tf.Tensor(-1.8482313, shape=(), dtype=float32)
VALID: 	 cnt: 127 	 Loss: tf.Tensor(-1.6852822, shape=(), dtype=float32)
VALID: 	 cnt: 255 	 Loss: tf.Tensor(-1.6392711, shape=(), dtype=float32)
VALID: 	 cnt: 383 	 Loss: tf.Tensor(-1.5989972, shape=(), dtype=float32)
VALID: 	 cnt: 511 	 Loss: tf.Tensor(-1.6133004, shape=(), dtype=float32)
VALID: 	 cnt: 639 	 Loss: tf.Tensor(-1.4430165, shape=(), dtype=float32)
VALID: 	 cnt: 767 	 Loss: tf.Tensor(-1.1516627, shape=(), dtype=float32)
Epoch:  4
TRAIN: 	 cnt: 127 	 Loss: tf.Tensor(-1.1793005, shape=(), dtype=float32)
TRAIN: 	 cnt: 255 	 Loss: tf.Tensor(

TRAIN: 	 cnt: 1023 	 Loss: tf.Tensor(-2.648402, shape=(), dtype=float32)
TRAIN: 	 cnt: 1151 	 Loss: tf.Tensor(-2.921479, shape=(), dtype=float32)
TRAIN: 	 cnt: 1279 	 Loss: tf.Tensor(-3.0468466, shape=(), dtype=float32)
TRAIN: 	 cnt: 1407 	 Loss: tf.Tensor(-3.20812, shape=(), dtype=float32)
TRAIN: 	 cnt: 1535 	 Loss: tf.Tensor(-3.4970891, shape=(), dtype=float32)
TRAIN: 	 cnt: 1663 	 Loss: tf.Tensor(-2.9355884, shape=(), dtype=float32)
TRAIN: 	 cnt: 1791 	 Loss: tf.Tensor(-3.0716422, shape=(), dtype=float32)
TRAIN: 	 cnt: 1919 	 Loss: tf.Tensor(-4.13975, shape=(), dtype=float32)
TRAIN: 	 cnt: 2047 	 Loss: tf.Tensor(-4.1034493, shape=(), dtype=float32)
TRAIN: 	 cnt: 2175 	 Loss: tf.Tensor(-3.7367582, shape=(), dtype=float32)
TRAIN: 	 cnt: 2303 	 Loss: tf.Tensor(-4.7711215, shape=(), dtype=float32)
TRAIN: 	 cnt: 2431 	 Loss: tf.Tensor(-3.072369, shape=(), dtype=float32)
TRAIN: 	 cnt: 2559 	 Loss: tf.Tensor(-4.282543, shape=(), dtype=float32)
TRAIN: 	 cnt: 2687 	 Loss: tf.Tensor(-3.803426

VALID: 	 cnt: 255 	 Loss: tf.Tensor(-3.5495763, shape=(), dtype=float32)
VALID: 	 cnt: 383 	 Loss: tf.Tensor(-4.1809115, shape=(), dtype=float32)
VALID: 	 cnt: 511 	 Loss: tf.Tensor(-3.7993672, shape=(), dtype=float32)
VALID: 	 cnt: 639 	 Loss: tf.Tensor(-2.9767208, shape=(), dtype=float32)
VALID: 	 cnt: 767 	 Loss: tf.Tensor(-1.9858203, shape=(), dtype=float32)
Epoch:  11
TRAIN: 	 cnt: 127 	 Loss: tf.Tensor(-2.6219137, shape=(), dtype=float32)
TRAIN: 	 cnt: 255 	 Loss: tf.Tensor(-2.6425402, shape=(), dtype=float32)
TRAIN: 	 cnt: 383 	 Loss: tf.Tensor(-3.7848177, shape=(), dtype=float32)
TRAIN: 	 cnt: 511 	 Loss: tf.Tensor(-5.212933, shape=(), dtype=float32)
TRAIN: 	 cnt: 639 	 Loss: tf.Tensor(-4.8981013, shape=(), dtype=float32)
TRAIN: 	 cnt: 767 	 Loss: tf.Tensor(-3.7803192, shape=(), dtype=float32)
TRAIN: 	 cnt: 895 	 Loss: tf.Tensor(-5.060372, shape=(), dtype=float32)
TRAIN: 	 cnt: 1023 	 Loss: tf.Tensor(-4.0590186, shape=(), dtype=float32)
TRAIN: 	 cnt: 1151 	 Loss: tf.Tensor(-4.4

TRAIN: 	 cnt: 1919 	 Loss: tf.Tensor(-5.2776327, shape=(), dtype=float32)
TRAIN: 	 cnt: 2047 	 Loss: tf.Tensor(-5.213925, shape=(), dtype=float32)
TRAIN: 	 cnt: 2175 	 Loss: tf.Tensor(-4.7902083, shape=(), dtype=float32)
TRAIN: 	 cnt: 2303 	 Loss: tf.Tensor(-5.2296796, shape=(), dtype=float32)
TRAIN: 	 cnt: 2431 	 Loss: tf.Tensor(-4.1662803, shape=(), dtype=float32)
TRAIN: 	 cnt: 2559 	 Loss: tf.Tensor(-5.00257, shape=(), dtype=float32)
TRAIN: 	 cnt: 2687 	 Loss: tf.Tensor(-5.0126214, shape=(), dtype=float32)
TRAIN: 	 cnt: 2815 	 Loss: tf.Tensor(-5.118327, shape=(), dtype=float32)
TRAIN: 	 cnt: 2943 	 Loss: tf.Tensor(-5.0703635, shape=(), dtype=float32)
TRAIN: 	 cnt: 3071 	 Loss: tf.Tensor(-4.915835, shape=(), dtype=float32)
TRAIN: 	 cnt: 3199 	 Loss: tf.Tensor(-4.790986, shape=(), dtype=float32)
VALID: 	 cnt: 127 	 Loss: tf.Tensor(-4.017537, shape=(), dtype=float32)
VALID: 	 cnt: 255 	 Loss: tf.Tensor(-3.6570008, shape=(), dtype=float32)
VALID: 	 cnt: 383 	 Loss: tf.Tensor(-4.2379756,

TRAIN: 	 cnt: 383 	 Loss: tf.Tensor(-3.884914, shape=(), dtype=float32)
TRAIN: 	 cnt: 511 	 Loss: tf.Tensor(-5.38912, shape=(), dtype=float32)
TRAIN: 	 cnt: 639 	 Loss: tf.Tensor(-5.077876, shape=(), dtype=float32)
TRAIN: 	 cnt: 767 	 Loss: tf.Tensor(-3.998054, shape=(), dtype=float32)
TRAIN: 	 cnt: 895 	 Loss: tf.Tensor(-5.210382, shape=(), dtype=float32)
TRAIN: 	 cnt: 1023 	 Loss: tf.Tensor(-4.342013, shape=(), dtype=float32)
TRAIN: 	 cnt: 1151 	 Loss: tf.Tensor(-4.636067, shape=(), dtype=float32)
TRAIN: 	 cnt: 1279 	 Loss: tf.Tensor(-4.6089044, shape=(), dtype=float32)
TRAIN: 	 cnt: 1407 	 Loss: tf.Tensor(-4.558968, shape=(), dtype=float32)
TRAIN: 	 cnt: 1535 	 Loss: tf.Tensor(-5.1229124, shape=(), dtype=float32)
TRAIN: 	 cnt: 1663 	 Loss: tf.Tensor(-3.6481228, shape=(), dtype=float32)
TRAIN: 	 cnt: 1791 	 Loss: tf.Tensor(-3.939966, shape=(), dtype=float32)
TRAIN: 	 cnt: 1919 	 Loss: tf.Tensor(-5.3766026, shape=(), dtype=float32)
TRAIN: 	 cnt: 2047 	 Loss: tf.Tensor(-5.293609, shape

## 6. Test
### 6.1 Loading Test Dataset

In [13]:
dset_test = TrajectoryDataset(
    data_set + 'test1/',
    obs_len=obs_seq_len,
    pred_len=pred_seq_len,
    skip=1)

loader_test = np.asarray(dset_test)

100%|█████████████████████████████████████████████████████████████████████████████| 818/818 [00:00<00:00, 10097.92it/s]

Processing Data .....


### 6.2 Testing

In [14]:
KSTEPS = 20
ade_ls = []
fde_ls = []
print('Number of samples:', KSTEPS)
print("*" * 50)

obs_seq_len = args.obs_len
pred_seq_len = args.pred_len
data_set = './dataset/' + args.dataset + '/'

ad_ = 999999
fd_ = 999999
print("Testing ....")
ade_,fde_,raw_data_dict = test(model, loader_test)
ade_ = min(ade_, ad_)
fde_ = min(fde_, fd_)
ade_ls.append(ade_)
fde_ls.append(fde_)
print("ade:", ade_, " fde:", fde_)

  0%|                                                                                          | 0/818 [00:00<?, ?it/s]

Number of samples: 20
**************************************************
Testing ....
Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


100%|████████████████████████████████████████████████████████████████████████████████| 818/818 [14:22<00:00,  1.05s/it]

ade: 0.14121786136942202  fde: 0.24519761377462826


### 6.3 Save Trojectory Data

In [15]:
def predict_result(model, loader_test, KSTEPS=20):
    raw_data_dict = {}
    ade_bigls = []
    fde_bigls = []
    pred_visual = []
    tragets_visual = []

    step =0
    for batch in loader_test:
        step += 1
        
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped, loss_mask, V_obs, V_tr = batch

        V_obs = V_obs[np.newaxis, :]
        V_tr = V_tr[np.newaxis, :]

        identity_spatial = tf.ones((V_obs.shape[1], V_obs.shape[2], V_obs.shape[2])) * tf.eye(V_obs.shape[2])
        identity_temporal = tf.ones((V_obs.shape[2], V_obs.shape[1], V_obs.shape[1])) * tf.eye(V_obs.shape[1])
        identity = [identity_spatial, identity_temporal]
        
        V_pred = model(V_obs, identity)
        V_tr = tf.cast(tf.squeeze(V_tr), dtype = tf.float32)
        
        num_of_objs = obs_traj_rel.shape[1]
        V_pred, V_tr = V_pred[:, :num_of_objs, :], V_tr[:, :num_of_objs, :]
        
        sx = tf.math.exp(V_pred[:,:,2]) #sx
        sy = tf.math.exp(V_pred[:,:,3]) #sy
        corr = tf.math.tanh(V_pred[:,:,4]) #corr
        
        cov = np.zeros((V_pred.shape[0],V_pred.shape[1],2,2))
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        cov = tf.convert_to_tensor(cov, dtype=tf.float32)
        mean = V_pred[:,:,0:2]
        
        mvnormal = tfp.distributions.MultivariateNormalFullCovariance(mean, cov)
        
        ade_ls = {}
        fde_ls = {}
        V_x = seq_to_nodes(tf.identity(obs_traj))
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_obs[:,:,:,:2])), tf.identity(V_x[0,:,:]))
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_tr)),tf.identity(V_x[-1,:,:]))
        
        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []
        
        for n in range(num_of_objs):
            ade_ls[n]=[]
            fde_ls[n]=[]
        
        for k in range(KSTEPS):

            V_pred = mvnormal.sample()

            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(tf.identity(tf.squeeze(V_pred)), tf.identity(V_x[-1,:,:]))

            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))

            for n in range(num_of_objs):
                pred = []
                target = []
                obsrvs = []
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)
                
                if n == 0:
                    pred_visual.append(pred)
                    tragets_visual.append(target)

                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))
                   
        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))

    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_, fde_, raw_data_dict, pred_visual, tragets_visual

def pre_and_result(pre_or_res = 0):
    frame_id_x_y = []
    for id, pedes in enumerate(zip(pred_visual, target_visual)):
        id_x_y = []
        frame_ids = []
    #print(pedes[0])
        frame_id = 0
        for pre in pedes[pre_or_res][0]:
      #print([item for item in pre])
      #print(pre[0])
            id_x_y.append([id+1, pre[0][0], pre[0][1]])
            frame_ids.append(frame_id)
            frame_id += 1
    #print(dict(zip(frame_ids, id_x_y)))
        frame_id_x_y.append(pd.DataFrame(dict(zip(frame_ids, id_x_y))))
        id += 1
    pre = frame_id_x_y[0]
    for item in frame_id_x_y[1:]:
        pre = pd.concat([pre, item], axis=1)
    return pre

In [16]:
# Predict trojectory
_, _, _, pred_visual, target_visual = predict_result(model, loader_test)

# Processing trojectory data
pre = pre_and_result(0)
res = pre_and_result(1)

# Save data
pre.to_csv('pre.csv')
res.to_csv('res.csv')